In [366]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for visualising

In [367]:
# read in all the words
words = open("names.txt", encoding="utf-8").read().splitlines()
words = [i.strip() for i in words if i]

In [368]:
len(words)

32033

In [369]:
# build the vocabulary of characters and mappings to /from integers
chars = sorted(list(set(''.join(words)))) # getting the character library
stoi = {val:key+1 for key, val in enumerate(chars)}
stoi['|'] = 0
itos = {ind:st for st, ind in stoi.items()}
print(itos)
lib_len = len(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '|'}


In [370]:
# building the dataset

block_size = 3 # context length: how many characters we take to predict the next one
dim_size = 10
X, Y = [], []

for w in words:
    # print(w)
    context = [0] * block_size

    for ch in w + '|':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [371]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [372]:
# building the dataset
def build_dataset(words):    
    block_size = 3 # context length: how many characters we take to predict the next one
    X, Y = [], []

    for w in words:
        # print(w)
        context = [0] * block_size

        for ch in w + '|':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
checkpoint_train = int(0.8 * len(words))
checkpoint_dev = int(0.9 * len(words))

X_train, Y_train = build_dataset(words[:checkpoint_train])
X_dev, Y_dev = build_dataset(words[checkpoint_train:checkpoint_dev])
X_test, Y_test = build_dataset(words[checkpoint_dev:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [373]:
h_neuron_count = 200
g = torch.Generator().manual_seed(412987498012)
C = torch.randn((lib_len, dim_size), generator=g)
W1 = torch.randn((block_size * dim_size, h_neuron_count), generator=g) # weights
b1 = torch.randn(h_neuron_count, generator=g) # biases
W2 = torch.randn((h_neuron_count, lib_len), generator=g) # weights for output layer
b2 = torch.randn(lib_len, generator=g) # biases for output layer
parameters = [C, W1, b1, W2, b2]

In [374]:
sum(p.nelement() for p in parameters) # parameter count

11897

In [375]:
for p in parameters:
    p.requires_grad = True

In [383]:
lre = torch.linspace(-3, 0, 1000) # 1000 numbers between -3 and 0 - equal distances
lrs = 10**lre
lri = []
loss_i = []
step_i = []
learning_rate = 0.01

In [384]:

for i in range(50000):
    # forward pass
    ix = torch.randint(0, X_train.shape[0], (34,))
    # print(ix.shape)

    emb = C[X_train[ix]]
    h = torch.tanh(emb.view(-1, block_size * dim_size) @ W1 + b1) # hidden layer
    logits = h @ W2 + b2
    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdims=True)
    # loss = -probs[torch.arange(len(Y)), Y].log().mean()
    loss = F.cross_entropy(logits, Y_train[ix]) # More readable and efficient way to calculate loss
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    # learning_rate = lrs[i]
    for p in parameters:
        p.data += -learning_rate * p.grad

    # stats tracking
    # lri.append(lre[i])
    step_i.append(i)
    loss_i.append(loss.log10().item())

print(loss.item())

2.291602849960327


In [ ]:
# plt.plot(step_i, loss_i)
# plt.plot(lri, loss_i)
C[X].shape, W1.shape

In [385]:
emb = C[X_train]
h = torch.tanh(emb.view(-1, block_size * dim_size) @ W1 + b1)
# print(h.shape)
logits = h @ W2 + b2
# print(logits.shape)
loss = F.cross_entropy(logits, Y_train)
print(loss.item())

2.105282783508301


In [386]:
emb = C[X_dev]
h = torch.tanh(emb.view(-1, block_size * dim_size) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y_dev)
print(loss.item())

2.1474034786224365


In [389]:
# generating names
g = torch.Generator().manual_seed(412987498012)

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[ch] for ch in out))

prig|
joha|
shaqi|
pomiil|
leyi|
zayah|
nachoralyina|
brys|
kenne|
jaysi|
mike|
kalanden|
itzariet|
arth|
kena|
iselizen|
josie|
dayse|
maau|
aria|


In [390]:
# generating names with considering name length frequency

word_lens = [len(w) for w in words]
word_lens = torch.tensor(word_lens)

wlenc = F.one_hot(word_lens, num_classes=max(word_lens).item() + 1)
lengths = wlenc.sum(0) # [n, n] -> [n] by adding all columns respectively

# frequency table of word lengths of data
len_frequency = lengths / lengths.sum()

In [391]:
w_lens = torch.multinomial(len_frequency, num_samples=20, replacement=True, generator=g)
for w_len in w_lens.data :
    out = []
    context = [0] * block_size
    i = w_len.item()
    while i != 0:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0 and i - 1 != 0:
            out = []
            context = [0] * block_size
            i = w_len.item()
        else : i -= 1

    print(''.join(itos[ch] for ch in out))

adelilie
duzail
gorbyn
nashu
chriend
ciylyn|
brihar
innio
khalin
jayla
salderenc
jorin
angelon
kmalaira
pailey|
corihan|
jacin
milia
ndhomi
deboaxe
